# PIA - Reinforcement Learning

1. RL Intro
2. Dynamic Programing
3. Tabular Q-Learning
4. **Deep Reinforcement Learning (DQN)**

## 4. Deep Reinforcement Learning (DQN)

Welcome to the final Reinforcement Learning Notebook. In this part you will implement the Deep Q-Learning Algorithm that was used by Mnih et al. to play Atari Video games. The resulting agent is called Deep Q-Network agent (or shorter DQN agent) because it uses a Deep Neural Network to approximate the value function (instead of saving it in a table).

---
### Course of Action

* Please write all executable python code in ```Code```-Cells (```Cell```->```Cell Type```->```Code```) and all Text as [Markdown](http://commonmark.org/help/) in ```Markdown```-Cells
* Describe your thinking and your decisions (where appropriate) in an extra Markdown Cell or via Python comments
* In general: discuss all your results and comment on them (are they good/bad/unexpected, could they be improved, how?, etc.). Furthermore, visualise your data (input and output).
* Write a short general conclusion at the end of the notebook
* Further experiments are encouraged. However, don't forget to comment on your reasoning.
* Use a scientific approach for all experiments (i.e. develop a hypothesis or concrete question, make observations, evaluate results)

### Submission

Upload the complete Notebook (and an exported html version) to Ilias before the deadline. One Notebook per Group is enough. Edit the teammember table below.

**Important**: Also attach a HTML version of your notebook (```File```->```Download as```->```HTML```) in addition to the ```.ipynb```-File.

| Teammember |                    |
|------------|--------------------|
| 1.         | Geoffrey Hinton    |
| 2.         | Yoshua Bengio      |
| 3.         | Yann LeCun         |
| 4.         | Jürgen Schmidhuber |
---

## Prerequisites

- **INSTALL**: [Gymnasium](https://github.com/Farama-Foundation/Gymnasium) - minimal install should be enough.
- The notebook is pretty self contained and includes all relevant theory. However, make sure that you have **read** and *understood* a corresponding lecture at least once! The notebook can only provide a short recap. Being prepared will help you to finish the notebook in time!
- Some good reading resources are:

    1. Johannes Maucher's [lecture on RL](https://gitlab.mi.hdm-stuttgart.de/maucher/ML/-/blob/master/Slides/V12Reinforcement.pdf) - chapter 4,5 & **6**.
    2. David Silver's course on RL: [Lecture 6](https://www.davidsilver.uk/teaching/): [Youtube](https://www.youtube.com/watch?v=UoPei5o4fps), [Lecture 9](https://www.davidsilver.uk/teaching/): [Youtube](https://www.youtube.com/watch?v=sGuiWX07sKw&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ&index=9).
    3. Sutton & Barto: [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html) chapter **16.5**, further reading and more details chapters 9,10 & 11 (complete online pdf draft available).
    4. Read the original DQN papers:
        - NIPS 2013 - [Playing Atari with Deep Reinforcement Learning](https://arxiv.org/abs/1312.5602)
        - Nature 2015 - [Human Level Control Through Deep Reinforcement Learning](https://deepmind.com/research/publications/human-level-control-through-deep-reinforcement-learning/)

---

## Theory

In the last notebook you have implemented the model-free **Q-Learning** algorithm and solved the *full reinforcement learning problem* by **learning from samples**. In this context, *full* refered to the fact that we dont have acess to the world model and *model-free* to the fact that we have not tried to learn that model. Furthermore, Q-Learning performed **online** updates to the policy, meaning that we have adjusted the policy *online* after every time step. Finally Q-Learning is an **off-policy** algorithm because we followed an e-greedy behavior policy while we have performed updates according to a greedy target policy. Now we will tackle the **curse of dimensionality** by **approximating the value function** instead of saving it explicitly in a table.

### Case Study - Video Games

<img src="./RL_pics/Atari_games.png" alt="DQN" style="width: 1000px;"/>

Before we proceed to the solution, let us quickly revise the actual problem that we are trying to solve. Consider the task of learning to play a video game given only the raw game screen as input. This is similar to how humans would play the game. Since the game screen is typically represented as raw pixels, this leaves us with a really high dimensional input or state space because every change of pixels represents a new and distinct state of the game, even if the change seems completely insignificant to you! Remember, the agent has no real knowledge of the game (or world model). Clearly it is infeasible to store every possible state of pixle combinations in a table. See [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html) chapter 16.5 for a comprehensive discussion.   

<img src="./RL_pics/DQN_principle.png" alt="DQN" style="width: 800px;"/>

> The problem that is solved by Deep Reinforcement Learning (in the case by DQN) is how to learn a mapping from a high dimensional input space to action values. This mapping represents the value function and can be used in a policy, e.g. to choose the best action with the highest value.


### Nonlinear Function Approximation with Artificial Neural Networks

First of all, a lookup table can mathematically be seen as a very simple form of a function, i.e. a direct mapping of values (hence the name value function). However, for the reasons explained above, this approach does not scale to high dimensional input spaces. A typical solution to this problem is to replace the *perfect* but intractable lookup table with a more complicated function that *only* **approximates the true value function** but is computationally tractable. In the case of DQN we choose a deep neural network as our function approximator. Formally, this new function is denoted as $\hat{Q}$ and we write

$$\begin{eqnarray} 
\hat{Q}(s,a,\theta) \approx Q_{\pi}(s,a)
\end{eqnarray}$$

where $\theta$ are the parameters of the neural network. In other words, the value function now depends on those parameters and the task of finding and optimal value function turns into the task of finding an optimal set of parameters for the network. Fortunately, we know how to train and optimize a neural network with SGD and backpropagation given an appropriate loss function. **Inside the RL-framework we can use the TD-error as the loss function**. Formally we optimize:  

$$\begin{eqnarray} 
L_i(\theta_{i}) = \Big( \underbrace{r_{t+1} + \gamma \max_a Q(s_{t+1}, a; \theta_i) - Q(s_t, a_t;\theta_i)}_{TD-error} \Big)^2
\end{eqnarray}$$

> Note that in order to obtain any action-values, we now need to perform a forward pass through the network. In practice, this means two forward passes before we can calculate the loss, one pass for the value of $Q(s_t, a_t;\theta_i)$ and another one to calculate the value of $\max_a Q(s_{t+1}, a; \theta_i)$. More details on that later.


### Instabilities and Solutions

So far, so good. By using the TD-error as loss function we can train the network in a supervised learning like setup. Sadly it is not that easy. Remember that in supervised learning we assumed the data to be *independent and identically distributed* (iid-data) in order for SGD to work properly. This assuption does not hold in reinforcement learning where subsequent data is highly correlated and in contrast, depends strongly on the agents last choice of actions. This inherent sequential property, in combination with an off-policy algorithm and a non-linear function appoximator such as a neural network, results in the problem that the learnable network parameters are at risk to oscillate or even diverge catastrophically during training. In theory, there is no convergence guarantee whatsoever. In practice, Mnih et al. found two major ways in which the training process can be stabilized:

- **Experience Replay** - This idea introduces a so called **replay buffer** $\mathcal{D}$ which stores the last $N$ state transitions as experience tuples $(S,A,R,S')$. In other words, the agent saves its recent history to a buffer. This way, experience can be reused and the correlation between samples can be broken by **drawing random minibatches** of experience $U(\mathcal{D})$ from it during the training.
<br><br>

- **Fixed Q-Targets** - The second idea is to keep **a separate set of parameters** $\theta^{−}$ for calculating the **TD target**. This set is basically a copy of $\theta$ that is held fixed for some time $t$ and periodically gets swapped with the current parameter values in order to allow progress. Mnih et al. have shown that updating $\theta$ towards such fixed Q-targets is another effective way to stabilize the training process. In practice, this means that we have basically two separate networks which we will distinguish by their different set of parameters $\theta^{-}$ and $\theta$. We will refer to them respectively as **Target-** and **Q-Network**.

As a result, the **Q-learning update** of DQN at iteration $i$ uses the following loss function: 

$$\begin{eqnarray} 
L_i(\theta_{i}) = \mathbb{E}_{(s,a,r,s') \sim U(\mathcal{D})} \Bigg[ \Bigg( r_{t+1} + \gamma \max_a \underbrace{Q(s_{t+1}, a; \theta^{-}_{i})}_{Target-Network} - \underbrace{Q(s_t, a_t;\theta_i)}_{Q-Network} \Big)^2 \Bigg]
\end{eqnarray}$$

And thats it! We can use this update rule inside the Q-Learning algorithm to train a Deep Q-Network with SGD as we do in supervised learning. The corresponding **Deep Q-Learning algorithm** is given in the next part.

---

## Implementation

As in the previous notebooks we will implement the DQN algorithm step by step. While the original DQN architecture was a CNN trained on Atari games, we will choose a much simpler problem and architecture. This way you can verify and debug your implementation much faster (in minutes vs hours...). However, the algorithm itself is still the same and extending it should be straightforward after completing the notebook. Though, this is left to the *further ideas part* depending on your time and motivation.

The following is an overview of all the parts you need. Use it as a checklist if you get lost. Like with Q-Learning, try first to verify that all the sub-parts are working as expected. If you are confident, integrate them iteratively into the main loop. There is no single best approach how to proceed so feel free to jump back and forth between the cells as you like.

##### Overview

- The Gymnasium Environment
- Replay Buffer
- Epsilon Schedule
- Deep Q-Network
- E-Greedy Policy (action selection)
- Update the Target Network
- Train Method
- Main Loop
- Evaluation of Deep Reinforcement Learning Algorithms



### The Open AI Gym Environment

You will use the Gymnasium environment to solve a classic control task known as **Cart Pole Balance**. The great thing about the gym environment is that it offers a common interface to [many different environments](https://gymnasium.farama.org/environments/classic_control/). That way you can easily test your algorithms on different tasks, e.g. switch from an easy one like CartPole to more challenging ones like an Atari game etc. ;)

For now, we will solve the **CartPole-v1** task. 

1. First of all, [go and read](https://gymnasium.farama.org/environments/classic_control/cart_pole/) about its most important details such as the *observations, actions, rewards,* it's *max length* etc.
2. Second, get used to the gym interface. Run a random agent for some episodes etc. The most important API calls are:
    - `gym.make('CartPole-v1')` returns a new `game`.
    - The `make` function also takes an argument [`render_mode=`](https://gymnasium.farama.org/api/env/#gymnasium.Env.render). You can set it to the string `'human'` if you want to see the agent play or `None` if you don't need it, e.g. during training.
    - The game's `action_space` and `observation_space` variables.
    - `reset()` - returns an initial `observation` and some debug/game `info`.
    - `step()`  - takes an `action` int, returns an `observation, reward, game_over, truncated, info` tuple.
    - `close()` - call this after the last episode has ended to clean up.

In [ ]:
import gymnasium as gym
import numpy as np

# You code comes here

### Replay Buffer

The replay buffer should store the last $N$ experience tuples. This is basically a FIFO queue and practically, [python offers such a data structure called](https://docs.python.org/3/library/collections.html#collections.deque) `deque`. If initialized with a `maxlen` parameter, `deque`'s `append` method will pop items from the left automatically when the list grows beyond the given `maxlen`. This is exactly what we want and **you can implement it in just a few lines of code!** The replay buffer should have the following methods:

- A constructor (`__init__`) that initializes an internal `deque` with a given `maxlen` or $N$ or better, call it `buffer_size`.

- A method `add` that appends a new experience tuple as list `[state, action, reward, next_state, done]` (`done` is the `game_over` information). 
  - **IMPORTANT:** cast the `reward` and `done` values with `np.float32(x)` to make keras happy later :)
- A method `sample` that returns random batchs of training data (of size `batch_size`). 
    1. You can use `random`'s `sample` method to get random experience tuples.
    2. The batch should be reshaped to separate batches: `[observations],[actions],[rewards]` etc. See the zip demo how you can achive this. Also, make them numpy arrays so keras is happy later, e.g. by using `map(np.array, )` on the zip output.
    3. Return the batches as a list. Later we will simply unpack it in the train method again.

Use the cells below to test your implementation, e.g. by filling it with some integers in a loop, check whats in the queue and test the sampling method etc.

In [ ]:
import random
from collections import deque

class ReplayBuffer():
    # You code comes here

In [ ]:
# zip *demo
mini_batch = [["o1","a1","r1"], ["o2","a2","r2"], ["o3","a3","r3"]]

for i in zip(*mini_batch):
    print(i)

In [ ]:
# TEST the replay buffer

### Epsilon Schedule

Last time, we calculated the current epsilon value inside the main loop. This time we want a little bit more control so let's create a class for that task. The reason for that is that we have to pre fill the replay buffer with some initial random experience before we can sample from it and start with the actual training. We want to control the amount of initial experience with a `pre_train_steps` variable. During this time, the schedule should return the `start_epsilon` value so that the agent behaves fully random. After that, the normal decay should be applied. The implementation needs two methods:

- `__init__` constructor, takes all hyper parameters for the schedule such as `start_epsilon, final_epsilon, pre_train_steps, final_exploration_step`, pre calculate the decay value per step here.
- `value` method, takes a time step `t` and returns a correpsonding `epsilon` value. If `t` is smaller or greater than the `pre_train_steps` or `final_exploration_step` return the fixed values accordingly. In between calculate the decayed `epsilon` value at time `t`.

Use the code in the cell below to test and visualize your schedule.

In [ ]:
class LinearSchedule():
    # You code comes here

In [ ]:
# TEST the epsilon schedule
import matplotlib.pyplot as plt

schedule    = # You code comes here
test_points = [schedule.value(t) for t in range(1100)]

plt.plot(test_points)

### Deep Q-Network

The original DQN agent included a CNN as shown in the theory part of this notebook. For our task however, a simple MLP with only one hidden layer should be enough. Starting that simple will help you to get other implementation details right. Later on you can easily scale up and switch the MLP for a more powerfull network.

The DQN algorithm requires us to keep two separate networks, namely, a main **Q-Network** and a second **Target-Network**. For that reason lets implement a method `DQN` that builds and returns a model with the keras functional API. The hidden layer should have a `relu` activation, the output layer should have a `linear` activation (these are the predicted Q-Values, don't threshold them). In the main loop set `num_hidden=64`.

In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

def DQN(name, input_size, hidden_size, output_size):
    # Your code comes here

In [ ]:
# TEST your model, e.g. Q_test = DQN(1,2,2), dummy_batch=np.array([[1]], Q_test(dummy_batch)

### E-Greedy Policy

As in the Q-Learning notebook, let us encapsulate the action selection into a separate method. This time however, selecting a greedy max action requires us to perform a forward pass through the Q-Network. The method per se remains as simple as in the Q-Learning case.

- Remeber to wrap a single observation from the environment in a fake batch `[]` and turn it into a numpy array.
- To get the index of the beste action you can use `tf.argmax`. Make sure to use the proper axis.
- Make sure to unpack the fake batch again. To get the actual value from a tensor you can call `.numpy()`.

In [ ]:
def choose_egreedy_action(Q, observation, epsilon, action_space):
    # Your code comes here

In [ ]:
# TEST the action selection of the model, e.g. test with epsilon=0 and print the Q-values before argmax.

### Update the Target Network

As explained in the theory part, the Target-Network will be fixed for some time steps $C$ while the main Q-Network gets update every training/update step. Every $C$ time steps however, we want to update the Target-Network with the latest information from the Q-Network. This basically means that we want to copy over all weights from the Q-Network and assign them to the Target-Network. The Q-Network itself remains unchanged. We will control this freeze frequency later inside the main loop and execute the copy process only every $C$ time steps.

The implementation for this is pretty simple in keras. You can use the `get_weights()` and `set_weights()` methods. Even though it is easy, make sure to test if everything works as intendet, e.g. print the network weights befor and after the call to `update_target`.

In [ ]:
def update_target(Q, T):
    # Your code comes here

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

# TEST the function by inspecting the weights of small dummy networks

### Q-Learning Calculations

So far, everything should have been more or less straightforward. Now we have to calculate the actuall loss and train the networks. Remember to use the Target-Network to calculate the **TD-Target** and the Q-Network to calculate the actual loss of the **TD-Error**. Here are the two equations we have to implement.  

##### TD-Target

$$\begin{eqnarray} 
y_i = r_{t+1} + \gamma \max_a \underbrace{Q(s_{t+1}, a; \theta^{-}_{i})}_{Target-Network}
\end{eqnarray}$$

Make sure to use `next_states` and not the `states` in this calculation. Also, we only want to consider Q-Values if the `next_state` was not terminal. We can do this with the `done` variable and a little trick. In the implementation, simply append `tf.abs(done - 1.0)` to the multiplication. If `done` was `True` this will result in a multiplication by `0` and we don't have to implement a nasty if statement. To get the max Q-Value you can use `tf.reduce_max` on the correct axis.

##### Loss (TD-Error)

Now that we have our targets $y_i$ we can calculate the the TD-Error and the mean square error as our loss function.

$$\begin{eqnarray} 
L_i(\theta_{i}) = \mathbb{E}_{(s,a,r,s') \sim U(\mathcal{D})} \Bigg[ \Bigg( y_i - \underbrace{Q(s_t, a_t;\theta_i)}_{Q-Network} \Big)^2 \Bigg]
\end{eqnarray}$$

Note that in this calculation we don't want to use the max Q-Values but the value from the action that was taken in this example. Again we can use a little trick and use a one-hot encoding of the historical actions to mask the output of the Q-Network. First use `tf.one_hot` with the actions to create the mask, then multiply it with the Q-Values and use `tf.reduce_sum` to get the values of the desired actions. Again, make sure to provide the proper axis. The rest is almost straigtforward. Calculate the loss on the td-error as mean squared error and use it in the code skeleton below to train the Q-Network.

### Train Method

Here is a code little skeleton to help you implement the loss and training procedue step by step. The scope `tf.GradientTape()` tracks the gradients of the loss calculation with respect to the model parameters. Finally we apply those gradients using the optimizer. If everything works you can comment in the `@tf.function` decorator which compiles the whole thing to a tensorflow operation which results in a significant speed up. Since it is hard to test this function manually, just include it in the main loop and react to errors or strange training behavior.

In [ ]:
#@tf.function
def train(Q, T, gamma, num_actions, optimizer, batch):
    
    # 1. unpack the values from batch (as returned by the ReplayBuffer), e.g. observations, actions, etc.

    # 2. calculate the td-target (code it step by step)
    td_target  = 
    
    # 3. calculate the td-error and the mean squared loss within the GradientTape.
    with tf.GradientTape() as tape:
        # Your code comes here (code it step by step)
        td_error = 
        loss     = 
    
    # 4. one update step
    grads = tape.gradient(loss, Q.trainable_weights)
    optimizer.apply_gradients(zip(grads, Q.trainable_weights))

### Main Loop

Now that we have all parts together, we can implement the main loop. Below is the pseudo code of the Deep Q-Learning Algorithm. The individual parts should look now familiar to you since we have implemented them step by step. Next we have to integrate all parts in the Agent-Environment loop. Ideally you start with a random agent and add the functions one by one. Remember to:

- set `observation = new_observation`
- train only after `t > pre_training_steps`. You also might want to control the training frequency.
- update the target network only after `t > pre_training_steps` and then only every $C$ steps.
- use the AdamOptimizer
- track the training progress by collecting the rewards, you might also calculate a running mean of the last 10 and 100 rewards
- from time to time, print the mean rewards to see the progress
- use the pseudo code as a guide if you get lost

---
##### Deep Q-Learning with experience replay

- Initialize replay memory $D$ to capacity $N$
- Initialize action-value function $Q$ with random weights $\theta$
- Initialize target action-value function $\hat{Q}$ with random weights $\theta^{-}$

- **For** $t = 1, T$ **do**
    - With probability $\epsilon$ select a random action $a_t$
    - otherwise select $a_t = \text{arg}\max_a Q(s_t,a;\theta)$
    <br><br>
    - Execute action $a_t$ in emulator and observe reward $r_t$ and state $s_{t+1}$
    - Store transition $(s_t,a_t,r_t,s_{t+1})$ in $D$
    - Sample random minibatch transitions $(s_j,a_j,r_j,s_{j+1})$ from $D$
    - set    
    $
    y_j = \begin{cases}
          r_j  & \text{if episode terminates at step } j + 1 \\               
          r_j + \gamma \max_a \hat{Q}(s_{j+1}, a; \theta^{-})  & \text{otherwise}
        \end{cases}
$
    
    - Perform a gradient descent step on $\big(y_j - Q(s_j,a_j;\theta)\big)^2$ with respect to the network parameters $\theta$
    - Every $C$ steps reset $\hat{Q} = Q$
- **End For**
---

Here are some default hyperparameters that should work fairly well. Training should run no longer than a few minutes even on a CPU.

- Epsilon Schedule: `start_epsilon=1, final_epsilon 0.02, pre_training_steps=1000, final_exploration_step=10000`
- Replay Buffer: `buffer_size=50000`
- Training: `T=40000, training_freq=1, update_feq=500, batch_size=32`
- Q-Learning: `gamma=0.99`
- DQN: `num_hidden=64`
- Optimizer: `learning_rate=0.001`


In [ ]:
# Set hyperparmeters here and initialize everything

# 1. Create a new game and save the dimensions of observations and actions
#    Note: Cast the action_space number to np.int32 here to make tf.one_hot happy...
# 2. Epsilon Schedule
# 3. Replay Buffer
# 4. Training hyperparameter
# 5. Q-Learning parameter
# 6. Create the DQN Networks
# 7. Create an optimizer instance
# 8. Track the total reward per episode in a list []

In [ ]:
# Implement the Agent-Environment loop here

### Plot statistics

Plot the reward per episode and potentially a running mean over 10 and 100 episodes to get smoother curves.

In [ ]:
# Your code comes here

### Evaluation of Deep Reinforcement Learning Algorithms

In general, the evaluation of deep RL is discussed controversially among researchers since it remains unclear how to benchmark and compare such algorithms properly. Is the return or average return a good performance measure? How big is the impact of hyperparameters vs. general algorithm vs. implementation etc.? See the paper [Deep Reinforcement Learning that Matters](https://arxiv.org/abs/1709.06560) from Henderson et al. 2017 for a nice overview of these problems.

As part of this notebook however, we will evaluate our algorithm as done by the authors of DQN. The testing is very simple. Let the trained agent play the game $30$ times with an e-greedy policy with a fixed $\epsilon = 0.05$ and report the average high score (return).

- Run the agent for 30 episodes with an `evaluation_epsilon = 0.05`.
- Plot or print the results in a decent way.

In [ ]:
# Your code comes here

## Further Ideas

- Experiment with different settings, e.g. extreme buffer sizes for instance.
    - How fast can you train the agent to a test score > 195? In other words, what is the smallest amount of training steps you need to achieve this goal?
    - Can you get a mean 100 test score > 199 / 299 / 399 / 499?
- Implement and experiment with new/different exploration schemes.
- Extend the algorithm to play other gym games or even Atari.

These are just ideas. As always, you can experiment with Hyperparameters/Network sizes but importantly, **reason about their effects etc.**!

## Playing Atari with Deep Reinforcement Learning

So far you have implemented a basic DQN agent. For simplicity we have left out some important details which are crucial in order to play video games. If you are eager to do this anyway here are the missing parts.

### Architecture

First of all, switch the simple MLP with the following architecture from the paper. Note that there are no pooling layers in this CNN!

<img src="./RL_pics/DQN_architecture.png" alt="DQN" style="width: 800px;"/>

### Observability and Preprocessing

We have not talked about observability so far. Formally, Atari video games are Partially Observable Markov Decision Processes or POMDPs. This means that the game screen is not a sufficient observation to fully describe the underlying state and that the Markov Assumption does not hold. A simple example to makes this clear. Think about the game Pong. Given only one frame, the agent has no way of telling if the ball is currently moving from left to right or from right to left. For that reason the authors used the last 4 frames of the game as observation. This turns the POMDP into an MDP again. Furthermore they applied some more preprocessing steps to the game screens such as turning them into gray scale, rescaling, and taking the max out of two subsequent frames. Please see [the methods section of the DQN paper](https://deepmind.com/research/publications/human-level-control-through-deep-reinforcement-learning/) for more details on that.

> Hint: In order to implement this you have to keep some sort of frame buffer of the last 4 frames etc.!

### Training Details

In order to play faster, the authors trained the network only every $K=4$ time step. In between, the last taken action was repeated. This allows to play more games, e.g. gather more experience in less time since stepping the emulator forward is computationally *cheaper* than training the network. Again see the paper for details.

During training, the authors clipped the reward to the range $R \in \{-1,0,1\}$. Remember to remove this constraint during testing again to get the *real high score*

Another way to improve training stability was to clip the gradients (or better the squared L2 loss) to the range -1,1. In other words, only apply L2 if the error is inside this range and take a linear loss outside. This corresponds to a Huber Loss. Here you can find a TensorFlow implementations of that from the [OpenAI baseline agents](https://github.com/openai/baselines/blob/24fe3d6576dd8f4cdd5f017805be689d6fa6be8c/baselines/common/tf_util.py#L39). Please read the [Wikipedia](https://en.wikipedia.org/wiki/Huber_loss) to see what the function is doing.

The original DQN implementation used a slightly modified version of RMSProp as its optimizer. You dont have to implement this. It is perfectly fine to stay with Adam for instance. However, be aware that the learning rate is a really crucial parameter in this context. If at least for any, this is definetly the first (and probably the most important) hyperparameter for which you want to test different settings!

### Hyper Parameters

See the paper for a list of good default parameters. Due to the lengthy training times you may want to reduce the total amount of time steps the agent will be trained. You also may adjust the exploration accordingly. However, be aware that exploration time is very important. You may want to benchmark a very short training first and then do some rough calculations of how long it will take to train the agent for some $x$ time steps etc. Then plan some experiments. 